In [1]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.interpolate import interp1d
from scipy import signal
import pickle
import re
from tqdm import tqdm
import matplotlib as mpl
%config InlineBackend.figure_format = 'retina'

In [2]:
shape_name = 'sphere'
data_surface = pd.read_csv(f'ansys/{shape_name}_fine_left_right_surf_nodes_xyz.csv',
                           header=1)
data_middle = pd.read_csv(f'ansys/{shape_name}_fine_middle_nodes_xyz.csv',
                          header=1)

In [3]:
data_surface.shape

(22, 33493)

In [4]:
data_middle.shape

(22, 55297)

In [5]:
data = pd.concat((data_surface, data_middle), axis=1, join='inner')
data

,Time,X-coordinate-10366,Time.1,X-coordinate-10367,Time.2,X-coordinate-10368,Time.3,X-coordinate-10369,Time.4,X-coordinate-10374,...,Z-coordinate-19577,Time.27644,Z-coordinate-19578,Time.27645,Z-coordinate-19579,Time.27646,Z-coordinate-19580,Time.27647,Z-coordinate-19581,Unnamed: 55296
0,0.000,0.005000,0.000,0.005000,0.000,0.005000,0.000,0.005000,0.000,0.005000,...,0.047489,0.000,0.047778,0.000,0.048047,0.000,0.048298,0.000,0.048528,NaN
1,0.055,0.005000,0.055,0.005000,0.055,0.005000,0.055,0.005000,0.055,0.005000,...,0.047490,0.055,0.047778,0.055,0.048048,0.055,0.048298,0.055,0.048528,NaN
2,0.110,0.005000,0.110,0.005000,0.110,0.005000,0.110,0.005000,0.110,0.005000,...,0.047490,0.110,0.047778,0.110,0.048048,0.110,0.048298,0.110,0.048528,NaN
3,0.165,0.005000,0.165,0.005000,0.165,0.005000,0.165,0.005000,0.165,0.005000,...,0.047490,0.165,0.047778,0.165,0.048048,0.165,0.048298,0.165,0.048528,NaN
4,0.220,0.005000,0.220,0.005000,0.220,0.005000,0.220,0.005000,0.220,0.005000,...,0.047490,0.220,0.047778,0.220,0.048048,0.220,0.048298,0.220,0.048528,NaN
5,0.275,0.005000,0.275,0.005000,0.275,0.005000,0.275,0.005000,0.275,0.005000,...,0.047492,0.275,0.047781,0.275,0.048050,0.275,0.048301,0.275,0.048530,NaN
6,0.330,0.005000,0.330,0.005000,0.330,0.005000,0.330,0.005000,0.330,0.005000,...,0.047494,0.330,0.047782,0.330,0.048052,0.330,0.048302,0.330,0.048532,NaN
7,0.385,0.005000,0.385,0.005000,0.385,0.005000,0.385,0.005000,0.385,0.005000,...,0.047497,0.385,0.047785,0.385,0.048054,0.385,0.048304,0.385,0.048534,NaN
8,0.440,0.005000,0.440,0.005000,0.440,0.005000,0.440,0.005000,0.440,0.005000,...,0.047500,0.440,0.047788,0.440,0.048058,0.440,0.048308,0.440,0.048537,NaN
9,0.495,0.005000,0.495,0.005000,0.495,0.005000,0.495,0.005000,0.495,0.005000,...,0.047502,0.495,0.047790,0.495,0.048059,0.495,0.048309,0.495,0.048539,NaN


In [6]:
data.shape[1] == data_surface.shape[1] + data_middle.shape[1]

True

In [7]:
len(data.columns)

88790

In [8]:
times = data.loc[:, ["Time"]]
assert(len(times) == data.shape[0])

In [9]:
coord_cols = [col for col in data.columns if "coordinate" in col]
coordinates = data.loc[:, coord_cols]

In [10]:
coord_npy = {}
for col in tqdm(coord_cols):
    coord_npy[col] = np.array(coordinates[col])

100%|███████████████████████████████████████████████████████████████████████████| 44394/44394 [01:02<00:00, 710.79it/s]


In [11]:
coordinates

,X-coordinate-10366,X-coordinate-10366,X-coordinate-10367,X-coordinate-10367,X-coordinate-10368,X-coordinate-10368,X-coordinate-10369,X-coordinate-10369,X-coordinate-10374,X-coordinate-10374,...,Z-coordinate-19577,Z-coordinate-19577,Z-coordinate-19578,Z-coordinate-19578,Z-coordinate-19579,Z-coordinate-19579,Z-coordinate-19580,Z-coordinate-19580,Z-coordinate-19581,Z-coordinate-19581
0,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047489,0.047489,0.047778,0.047778,0.048047,0.048047,0.048298,0.048298,0.048528,0.048528
1,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047490,0.047490,0.047778,0.047778,0.048048,0.048048,0.048298,0.048298,0.048528,0.048528
2,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047490,0.047490,0.047778,0.047778,0.048048,0.048048,0.048298,0.048298,0.048528,0.048528
3,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047490,0.047490,0.047778,0.047778,0.048048,0.048048,0.048298,0.048298,0.048528,0.048528
4,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047490,0.047490,0.047778,0.047778,0.048048,0.048048,0.048298,0.048298,0.048528,0.048528
5,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047492,0.047492,0.047781,0.047781,0.048050,0.048050,0.048301,0.048301,0.048530,0.048530
6,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047494,0.047494,0.047782,0.047782,0.048052,0.048052,0.048302,0.048302,0.048532,0.048532
7,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047497,0.047497,0.047785,0.047785,0.048054,0.048054,0.048304,0.048304,0.048534,0.048534
8,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047500,0.047500,0.047788,0.047788,0.048058,0.048058,0.048308,0.048308,0.048537,0.048537
9,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,...,0.047502,0.047502,0.047790,0.047790,0.048059,0.048059,0.048309,0.048309,0.048539,0.048539


In [12]:
indices = sorted(list(set([int(re.findall('(\d+)', col)[0]) for col in coord_cols])))
len(indices), len(coord_cols)//3

(13262, 14798)

In [13]:
def get_scalar(time, key):
    val = coord_npy[key][time]
    if isinstance(val, float):
        return val
    return val[0]

    try:
        val = coordinates.loc[time, key]
    except:
        print(key)
        val = np.nan
    if isinstance(val, float):
        return val
    if isinstance(val, np.ndarray):
        # all values must be equal
        val = [v for v in val if not np.isnan(v)]
        assert(len(set(val)) == 1)
        return val[0]
    if isinstance(val, pd.core.series.Series):
        return val[0]
    print("Invalid value:", type(val))
    raise Exception()

In [14]:
node_positions = np.zeros((len(times), len(indices), 3))
for time in tqdm(range(len(times))):
    for i, idx in enumerate(indices):
        node_positions[time, i, 0] = get_scalar(time, f'X-coordinate-{idx}')
        node_positions[time, i, 1] = get_scalar(time, f'Y-coordinate-{idx}')
        node_positions[time, i, 2] = get_scalar(time, f'Z-coordinate-{idx}')

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 30.34it/s]


In [25]:
np.savez(f'ansys/{shape_name}_node_xyz.npz',
         node_positions=node_positions,
         times=times,
         indices=indices)

In [23]:
times.to_numpy()[:,0]

array([0.        , 0.05499969, 0.10999968, 0.16499968, 0.21999976,
       0.27499986, 0.32999983, 0.38499975, 0.43999988, 0.49499995,
       0.54999977, 0.6049999 , 0.65999967, 0.71499968, 0.76999998,
       0.82499975, 0.87999988, 0.93499976, 0.98999965, 1.04499972,
       1.09999979, 1.10000014])

In [20]:
list(enumerate(times))

[(0, 'Time'), (1, 'Time')]